# Defining a partition key

Partition keys are used in Apache Kafka to determine which partition a message should be written to. This ensures that related messages are kept together in the same partition, which can be useful for ensuring order or for grouping related messages together for efficient processing. Additionally, partitioning data across multiple partitions allows Kafka to distribute load across multiple brokers and scale horizontally, while replicating data across multiple brokers provides fault tolerance.

You can define your partition keys when using the `@produces` decorator, this guide will demonstrate to you this feature.

In [ ]:
# Define producer with partition key mechanism

In [ ]:
# Kafka consumer read msgs and print partition key: msg.key to show that it has been set